In [2]:
import numpy as np
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px
from EC_data_processing_lib import get_header_length
from EC_data_processing_lib import analyse_file
from EC_data_processing_lib import get_decimal_separator
from pandas import DataFrame as df
from Data_setup_Lares_Oxides import data_set
import os
from galvani import BioLogic
fig = go.Figure()
colors = px.colors.qualitative.Safe


for i in data_set:
    file = data_set[i]['data_CP']
    if not os.path.exists(file):
        print("File not found: " + file)
        continue
    if file.endswith('mpt'):
        headerlength = get_header_length(file)
        data = pd.read_csv(file, encoding='ansi', sep="\t", decimal=get_decimal_separator(file),  skiprows=range(headerlength))
        current_column = '<I>/mA'
    else:
        mpr_file = BioLogic.MPRfile(file)
        data = pd.DataFrame(mpr_file.data)
        current_column = 'I/mA'
    voltage = data['<Ewe>/V']
    time = data['time/s']/60  #converts seconds to minutes
    time = time - time[0] #sets intitial time to 0
    endtime = time[-1:]
    print(endtime)
    voltage_corrected = data['<Ewe>/V'] - data['I/mA'] * data_set[i]['Ru']/1000 - data_set[i]['cal'] #does the iR correction converts to overpotential
    voltage_corrected = voltage_corrected*1000
    
    j = 0
    Final_Values = []
    # calculates the final overpotential values and prints them along with standard deviation    
    for j in (time[time>115]):
        final_η = df.mean(voltage_corrected)
        final_η = np.round(final_η)
        final_η_error = df.std(voltage_corrected)
        final_η_error = np.round(final_η_error)
        Final_Values = ([data_set[i]['label'], float(final_η), float(final_η_error)]) 
    Final_Values = np.array(Final_Values)
    
    print(Final_Values)

    fig.add_trace(go.Scatter(
        x=time,
        y=voltage_corrected,
        name= data_set[i]['label'],
        line_color = colors[data_set[i]['color_index']],
    )
    
    
    
    )
fig.update_xaxes(range=[0, 120],
    nticks=10,
    ticks='outside',
    showgrid=False, gridwidth=1, gridcolor='grey',
    zeroline=False, zerolinecolor='grey', zerolinewidth= 1,
    showline=True, linewidth=2, linecolor='black')

fig.update_yaxes(range=[300, 2000],
    nticks=10,
    showgrid=False, gridwidth=1, gridcolor='grey',
    ticks='outside',
    zeroline=True, zerolinecolor='grey', zerolinewidth= 1,
    showline=True, linewidth=2, linecolor='black')
fig.update_layout(
    #title="CV",
    xaxis_title="Time (min)",
    yaxis_title="Overpotential, η(mV) - iR<sub>u</sub>",
    font=dict(
        family="Times New Roman, monospace",
        size=18,
        color="dimgrey"
        
    )
)

fig.update_layout({
    "plot_bgcolor": "rgba(0, 0, 0, 0)",
    'paper_bgcolor': 'rgba(0, 0, 0, 0)', #sets the background to be transparent... might appear black in .jpeg but will be transparent in the .svg
})
fig.update_layout(
    width = 500,
    height = 500)
fig.show()
fig.write_image('Chronopotentiometry.jpeg')
fig.write_image('Chronopotentiometry.svg')


KeyError: '<Ewe>/ V'